In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
%load_ext autoreload
%autoreload
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from datasets.kitti_raw_monosf import KITTI_Raw_KittiSplit_Train, KITTI_Raw_KittiSplit_Valid
from models.CNet import CNet
from augmentations import Augmentation_SceneFlow

# args
args = {
    'batch_size' : 1,
    'num_workers' : 2,
    'epochs': 1, 
    'lr': 2e-4,
    'momentum': 0.9,
    'beta': 0.999,
    'weight_decay': 0.0,
    'train': True,
    'cuda': True
}

In [3]:
data_root = '/external/datasets/kitti_data_jpg/'
train_dataset = KITTI_Raw_KittiSplit_Train(args={}, root=data_root)
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=True, pin_memory=True)
# val_dataset = KITTI_Raw_KittiSplit_Valid(args={}, root=data_root)
# val_loader = DataLoader(val_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=False, pin_memory=True)

In [4]:
%autoreload 

augmentation = Augmentation_SceneFlow(args)
model = CNet(args)

if args['cuda']:
    augmentation = augmentation.cuda()
    model = model.cuda()

params = model.parameters()

optimizer = Adam(params, lr=args['lr'], betas=[args['momentum'], args['beta']], weight_decay=args['weight_decay'])

81 256


In [5]:
# DEBUG
import pynvml
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total / (1024**3)} Gb')
print(f'free     : {info.free / (1024**3)} Gb')
print(f'used     : {info.used / (1024**3)} Gb')

total    : 7.92620849609375 Gb
free     : 6.02313232421875 Gb
used     : 1.903076171875 Gb


In [7]:
%autoreload
for data_dict in train_loader:
    # Get input and target tensor keys
    input_keys = list(filter(lambda x: "input" in x, data_dict.keys()))
    target_keys = list(filter(lambda x: "target" in x, data_dict.keys()))
    tensor_keys = input_keys + target_keys

    # Possibly transfer to Cuda
    if args['cuda']:
        for key, value in data_dict.items():
            if key in tensor_keys:
                data_dict[key] = value.cuda(non_blocking=True)
            
    if augmentation is not None:
        with torch.no_grad():
            data_dict = augmentation(data_dict)
        
    x = model(data_dict)
    break
    

agumentation
total    : 7.92620849609375 Gb
free     : 2.77191162109375 Gb
used     : 5.154296875 Gb
DEBUG correlation: torch.Size([1, 256, 4, 13]) torch.Size([1, 256, 4, 13])
[81, 256, 64, 3, 3, 1, 1]
sceneflow util
total    : 7.92620849609375 Gb
free     : 2.77191162109375 Gb
used     : 5.154296875 Gb
sceneflow util
total    : 7.92620849609375 Gb
free     : 2.77191162109375 Gb
used     : 5.154296875 Gb
DEBUG correlation: torch.Size([1, 192, 8, 26]) torch.Size([1, 192, 8, 26])
DEBUG l>1: torch.Size([1, 81, 8, 26]) torch.Size([1, 192, 8, 26]) torch.Size([1, 32, 4, 13]) torch.Size([1, 3, 4, 13]) torch.Size([1, 1, 8, 26])
[81, 192, 64, 3, 3, 1, 1, 1]
sceneflow util
total    : 7.92620849609375 Gb
free     : 2.77191162109375 Gb
used     : 5.154296875 Gb
sceneflow util
total    : 7.92620849609375 Gb
free     : 2.77191162109375 Gb
used     : 5.154296875 Gb
DEBUG correlation: torch.Size([1, 128, 16, 52]) torch.Size([1, 128, 16, 52])
DEBUG l>1: torch.Size([1, 81, 16, 52]) torch.Size([1, 128, 1

In [11]:
del x
del data_dict
torch.cuda.empty_cache()

In [13]:
from augmentations import Augmentation_ScaleCrop

a = Augmentation_ScaleCrop(None)
a.calculate_tform_and_grids()

In [55]:
def train_one_epoch(model, dataloader, optimizer, augmentations):

  model.train()

  for i, data in enumerate(dataloader):
    continue

  return

In [58]:
for epoch in range(args['epochs']):
    loss_one_epoch = train_one_epoch(model, dataloader, optimizer, augmentations=augmentation)
    optimizer.zero_grad()
    loss_one_epoch.backward()
    

NameError: name 'optimizer' is not defined

In [1]:
import torch
import torch.nn as nn

def conv(in_planes, out_planes, kernel_size=3, stride=1, dilation=1, isReLU=True):
    if isReLU:
        return nn.Sequential(
            nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, dilation=dilation,
                      padding=((kernel_size - 1) * dilation) // 2, bias=True),
            nn.LeakyReLU(0.1, inplace=True)
        )
    else:
        return nn.Sequential(
            nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, dilation=dilation,
                      padding=((kernel_size - 1) * dilation) // 2, bias=True)
        )
    
class FeatureExtractor(nn.Module):
    def __init__(self, num_chs):
        super(FeatureExtractor, self).__init__()
        self.num_chs = num_chs
        self.convs = nn.ModuleList()

        for _, (ch_in, ch_out) in enumerate(zip(num_chs[:-1], num_chs[1:])):
            layer = nn.Sequential(
                conv(ch_in, ch_out, stride=2),
                conv(ch_out, ch_out)
            )
            self.convs.append(layer)

    def forward(self, x):
        feature_pyramid = []
        for conv in self.convs:
            x = conv(x)
            feature_pyramid.append(x)

        return feature_pyramid[::-1]

class MaskNetDecoder(nn.Module):
  def __init__(self, in_ch, num_refs=1):
    super(MaskNetDecoder, self).__init__()

    conv_chs = [256, 256, 128, 64, 32, 16]
    self.convs = nn.Sequential(
      conv(in_ch,       in_ch,       kernel_size=7),
      conv(in_ch,       conv_chs[0], kernel_size=5),
      conv(conv_chs[0], conv_chs[1]),
      conv(conv_chs[1], conv_chs[2]),
      conv(conv_chs[2], conv_chs[3]),
      conv(conv_chs[3], conv_chs[4]),
      conv(conv_chs[4], conv_chs[5]),
    )

    self.pred_mask = conv(conv_chs[-1], num_refs)
    self.pred_mask_upconv = upconv_transpose(num_refs, num_refs)
    self.refine_upconv = conv(num_refs, num_refs)
    
  def forward(self, pyr_feat, nlevel):
    out_conv = self.convs(pyr_feat)
    mask = self.pred_mask(out_conv)
    pred_mask = nn.functional.sigmoid(mask)
    mask_upconv = self.pred_mask_upconv(mask)
    pred_mask_upconv = nn.functional.sigmoid(self.refine_upconv(mask_upconv))

    return pred_mask, pred_mask_upconv  # (B, num_refs, H, W), (B, num_refs, H*2, W*2)    


convs = nn.ModuleList()
mask_decoders = nn.ModuleList()

num_chs = [3, 32, 64, 96, 128, 192, 256]
search_range = 4
output_level = 4
num_levels   = 7

for l, ch in enumerate(num_chs[::-1]):
    if l > output_level:
                break
    if l == 0:
        num_ch_in = dim_corr + ch 
    else:
        num_ch_in = dim_corr + ch + 32 + 3 + 1
    print(num_ch_in)
    
x = torch.rand(256, 832, 3).permute(2, 0, 1).unsqueeze(0)
feat_extractor = FeatureExtractor(num_chs)
pyr = feat_extractor(x)
print([_.shape for _ in pyr] + [x.shape])

NameError: name 'dim_corr' is not defined